<a href="https://colab.research.google.com/github/Aqila-af/Aqila-af/blob/main/Workstation_Cluster.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get update

# Install Java 17 JDK
!apt-get install openjdk-17-jdk

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Ign:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [3,108 kB]
Get:14 http://archive.u

In [2]:
!git clone https://github.com/quasylab/sibilla

Cloning into 'sibilla'...
remote: Enumerating objects: 24338, done.
remote: Counting objects: 100% (7192/7192), done.
remote: Compressing objects: 100% (1898/1898), done.
remote: Total 24338 (delta 2556), reused 6991 (delta 2387), pack-reused 17146 (from 1)
Receiving objects: 100% (24338/24338), 11.27 MiB | 18.63 MiB/s, done.
Resolving deltas: 100% (9409/9409), done.


In [3]:
!cd sibilla && ./gradlew build -x test && ./gradlew installDist
!cp -a sibilla/shell/src/dist/scripts/sibilla_py .
!cd sibilla_py && pip install .

.............10%.............20%.............30%.............40%.............50%.............60%..............70%.............80%.............90%.............100%

Welcome to Gradle 8.8!

Here are the highlights of this release:
 - Running Gradle on Java 22
 - Configurable Gradle daemon JVM
 - Improved IDE performance for large projects

For more details see https://docs.gradle.org/8.8/release-notes.html

Starting a Gradle Daemon (subsequent builds will be faster)


> Starting Daemon> Starting Daemon > Connecting to Daemon> IDLE<-------------> 0% INITIALIZING s]<-------------> 0% INITIALIZING s]<-------------> 0% INITIALIZING s]<-------------> 0% INITIALIZING s]<-------------> 0% INITIALIZING s]<-------------> 0% INITIALIZING s]<-------------> 0% INITIALIZING s]<-------------> 0% INITIALIZING s]<-------------> 0% INITIALIZING s]<-------------> 0% INITIALIZING [1s]<-------------> 0% INITIALIZING [2s]> Evaluating settings<-------------> 0% INITIALIZING [3s]<-------------> 0% INITIALIZING

In [ ]:
import os
os.environ["SSHELL_PATH"]="/content/sibilla/shell/build/install/sshell/"
import sibilla


- - - - - - - - - - - - - - - - - - - - -

- - - - - -  SIBILLA IMPORTED - - - - - -

- - - - - - - - - - - - - - - - - - - - -



# **WorkStation Cluster**

This case study is based on a cluster of workstations.

The system comprises two sub-clusters with N workstations in each, connected in a star topology. The switches connecting each sub-cluster are joined by a central backbone. All components can break down and there is a single repair unit to service all components.

Study under which conditions the system guarantees that the following Quality of Service (QoS) levels are guaranteed:

minimum QoS: at least 3N/4 workstations are operational and connected via switches and backbone;

premium QoS: at least N workstations are operational and connected via switches and backbone.

In [ ]:
%%writefile workstation_cluster.pm

param N = 5;
param mf = 120;
param mr = 40;
param S = 2;

species WorkstationActive;
species WorkstationFailed;
species SwitchActive;
species SwitchFailed;
species BackboneActive;
species BackboneFailed;
species RepairUnitFree;
species RepairUnitBusy;


rule workstation_fails {
    WorkstationActive -[1.0/mf]-> WorkstationFailed
}

rule switch_fails {
    SwitchActive -[1.0/mf]-> SwitchFailed | WorkstationFailed
}

rule backbone_fails {
    BackboneActive -[1.0/mf]-> BackboneFailed | SwitchFailed
}

rule start_repair_process {
    WorkstationFailed | SwitchFailed | BackboneFailed | RepairUnitFree -[1.0/mr]-> RepairUnitBusy | WorkstationFailed | SwitchFailed | BackboneFailed
}

rule backbone_repair {
    RepairUnitBusy | BackboneFailed -[1.0/mr]-> RepairUnitFree | BackboneActive
}

rule repair_switch {
    RepairUnitBusy | SwitchFailed -[1.0/mr]-> RepairUnitFree | SwitchActive
}

rule repair_workstation {
    RepairUnitBusy | WorkstationFailed -[1.0/mr]-> RepairUnitFree | WorkstationActive
}

measure MinQoS = 3*(#WorkstationActive)/4;
measure PremQoS = #WorkstationActive;
measure active_workstations = #WorkstationActive;
measure active_switches = #SwitchActive;
measure active_backbone = #BackboneActive;
measure repair_units_busy = #RepairUnitBusy;
measure repair_units_free = #RepairUnitFree;

predicate minimumQoS = ((#WorkstationActive >= 3*N/4) && (#SwitchActive == S) && (#BackboneActive == 1));
predicate premiumQoS = ((#WorkstationActive >= N) && (#SwitchActive == S) && (#BackboneActive == 1));


system init =
    WorkstationActive<2*N> |
    WorkstationFailed<0> |
    SwitchActive<2> |
    SwitchFailed<0> |
    BackboneActive<1> |
    BackboneFailed<0> |
    RepairUnitFree<1> |
    RepairUnitBusy<0>;


Overwriting workstation_cluster.pm


In [ ]:
sibilla_runtime = sibilla.SibillaRuntime()
sibilla_runtime.load_module("population")
sibilla_runtime.load_from_file("workstation_cluster.pm")
sibilla_runtime.set_configuration("init")
sibilla_runtime.set_deadline(1440)
sibilla_runtime.set_dt(1)
sibilla_runtime.set_replica(100)

In [ ]:
sibilla_runtime.remove_all_measures()
sibilla_runtime.add_measure("MinQoS")
sibilla_runtime.add_measure("PremQoS")
sibilla_runtime.add_measure("active_backbone")
sibilla_runtime.add_measure("active_switches")
sibilla_runtime.add_measure("active_workstations")
sibilla_runtime.add_measure("repair_units_busy")
sibilla_runtime.add_measure("repair_units_free")

In [ ]:
simulation_result = sibilla_runtime.simulate("base_model")
simulation_result.results = {"#"+k : v for k,v in simulation_result.results.items()}
simulation_result.results.keys()

 The simulation has been successfully completed


dict_keys(['#MinQoS', '#PremQoS', '#active_backbone', '#active_switches', '#active_workstations', '#repair_units_busy', '#repair_units_free'])

In [ ]:
simulation_result.plot(show_sd = True)


In [ ]:
reach_minQoS = sibilla_runtime.evaluate_reachability("premiumQoS", delta=0.1, epsilon= 0.01)
print(reach_minQoS)

 The reachability evaluation has been successfully completed

Probability of reaching premiumQoS is 
0.9509433962264151 

error prob (epsilon) :  0.01
error gap  (delta)   :  0.1



In [ ]:
reach_premQoS = sibilla_runtime.evaluate_reachability("minimumQoS", delta=0.1, epsilon= 0.01)
print(reach_premQoS)

 The reachability evaluation has been successfully completed

Probability of reaching minimumQoS is 
0.9509433962264151 

error prob (epsilon) :  0.01
error gap  (delta)   :  0.1



In [ ]:
print('Simulation took:')
print(str(simulation_result.time_enlapsed) + 'second')
print('And it used:')
print(str(simulation_result.memory_used) + 'MiB')

Simulation took:
1.0344421879999999second
And it used:
1.54296875MiB


In [ ]:
sibilla_runtime.clear()

# **Possible Solutions**

**Higher mean time between failures (MTBF) on QoS of the network**

In [ ]:
with sibilla_runtime as sr:
  sr.load_module("population")
  sr.load_from_file("workstation_cluster.pm")
  sr.set_configuration("init")
  sr.set_deadline(1440)
  sr.set_dt(1)
  sr.set_replica(100)

  # get all the aviable parameters name
  print('all aviable parameters name : ')
  print(sr.get_parameters())

  sr.remove_all_measures()
  sibilla_runtime.add_measure("MinQoS")
  sibilla_runtime.add_measure("PremQoS")

  sr.set_parameter("mf", 480, True, False)
  sr.set_configuration("init")

  print(sr.get_parameter('N'))
  print(sr.get_parameter('mf'))
  print(sr.get_parameter('mr'))
  print(sr.get_parameter('S'))

  simulation_result = sr.simulate("High_MTBF")

  simulation_result.results = {"#"+k : v for k,v in simulation_result.results.items()}

  simulation_result.plot(show_sd = True)

  print(sr.evaluate_reachability("minimumQoS", delta=0.1, epsilon= 0.01))
  print(sr.evaluate_reachability("premiumQoS", delta=0.1, epsilon= 0.01))

all aviable parameters name : 
['N', 'S', 'mf', 'mr']
mf changed to 480current configuration : init
5.0
480.0
40.0
2.0
 The simulation has been successfully completed


 The reachability evaluation has been successfully completed

Probability of reaching minimumQoS is 
0.630188679245283 

error prob (epsilon) :  0.01
error gap  (delta)   :  0.1

 The reachability evaluation has been successfully completed

Probability of reaching premiumQoS is 
0.6 

error prob (epsilon) :  0.01
error gap  (delta)   :  0.1



In [ ]:
with sibilla_runtime as sr:
  sr.load_module("population")
  sr.load_from_file("workstation_cluster.pm")
  sr.set_configuration("init")
  sr.set_deadline(1440)
  sr.set_dt(1)
  sr.set_replica(100)

  # get all the aviable parameters name
  print('all aviable parameters name : ')
  print(sr.get_parameters())

  sr.remove_all_measures()
  sibilla_runtime.add_measure("MinQoS")
  sibilla_runtime.add_measure("PremQoS")

  sr.set_parameter("N", 30, True, False)
  sr.set_parameter("mf", 10, True, False)
  sr.set_parameter("mr", 40, True, False)
  sr.set_configuration("init")

  print(sr.get_parameter('N'))
  print(sr.get_parameter('mf'))
  print(sr.get_parameter('mr'))
  print(sr.get_parameter('S'))

  simulation_result = sr.simulate("optimum_parameters")
  simulation_result.results = {"#"+k : v for k,v in simulation_result.results.items()}
  simulation_result.plot(show_sd = True)
  print(sr.evaluate_reachability("minimumQoS", delta=0.1, epsilon= 0.01))
  print(sr.evaluate_reachability("premiumQoS", delta=0.1, epsilon= 0.01))


all aviable parameters name : 
['N', 'S', 'mf', 'mr']
N changed to 30current configuration : init
mf changed to 10current configuration : init
mr changed to 40current configuration : init
30.0
10.0
40.0
2.0
 The simulation has been successfully completed


 The reachability evaluation has been successfully completed

Probability of reaching minimumQoS is 
1.0 

error prob (epsilon) :  0.01
error gap  (delta)   :  0.1

 The reachability evaluation has been successfully completed

Probability of reaching premiumQoS is 
1.0 

error prob (epsilon) :  0.01
error gap  (delta)   :  0.1



In [ ]:
sibilla_runtime.clear()

**Fast Repair: Reducing the time it takes to repair failed components.**

In [ ]:
with sibilla_runtime as sr:
  sr.load_module("population")
  sr.load_from_file("workstation_cluster.pm")
  sr.set_configuration("init")
  sr.set_deadline(1440)
  sr.set_dt(1)
  sr.set_replica(100)

  # get all the aviable parameters name
  print('all aviable parameters name : ')
  print(sr.get_parameters())

  sr.remove_all_measures()
  sibilla_runtime.add_measure("MinQoS")
  sibilla_runtime.add_measure("PremQoS")

  sr.set_parameter("mr", 30, True, False)
  sr.set_parameter("mf", 480, True, False)
  sr.set_configuration("init")

  print(sr.get_parameter('N'))
  print(sr.get_parameter('mf'))
  print(sr.get_parameter('mr'))
  print(sr.get_parameter('S'))

  simulation_result = sr.simulate("Fast_repair_time")
  simulation_result.results = {"#"+k : v for k,v in simulation_result.results.items()}
  simulation_result.plot(show_sd = True)
  print(sr.evaluate_reachability("minimumQoS", delta=0.1, epsilon= 0.01))
  print(sr.evaluate_reachability("premiumQoS", delta=0.1, epsilon= 0.01))

all aviable parameters name : 
['N', 'S', 'mf', 'mr']
mr changed to 30current configuration : init
mf changed to 480current configuration : init
5.0
480.0
30.0
2.0
 The simulation has been successfully completed


 The reachability evaluation has been successfully completed

Probability of reaching minimumQoS is 
0.5547169811320755 

error prob (epsilon) :  0.01
error gap  (delta)   :  0.1

 The reachability evaluation has been successfully completed

Probability of reaching premiumQoS is 
0.5849056603773585 

error prob (epsilon) :  0.01
error gap  (delta)   :  0.1



In [ ]:
sibilla_runtime.clear()

**Redundancy: Backup components that activate immediately when a failure occurs.**

In [ ]:
with sibilla_runtime as sr:
  sr.load_module("population")
  sr.load_from_file("workstation_cluster.pm")
  sr.set_configuration("init")
  sr.set_deadline(1440)
  sr.set_dt(1)
  sr.set_replica(100)

  # get all the aviable parameters name
  print('all aviable parameters name : ')
  print(sr.get_parameters())

  sr.remove_all_measures()
  sibilla_runtime.add_measure("MinQoS")
  sibilla_runtime.add_measure("PremQoS")

  sr.set_parameter("N", 20, True, False)
  sr.set_parameter("mf", 480, True, False)
  sr.set_parameter("mr", 30, True, False)
  sr.set_configuration("init")

  print(sr.get_parameter('N'))
  print(sr.get_parameter('mf'))
  print(sr.get_parameter('mr'))
  print(sr.get_parameter('S'))

  simulation_result = sr.simulate("redandent_component")
  simulation_result.results = {"#"+k : v for k,v in simulation_result.results.items()}
  simulation_result.plot(show_sd = True)

  reach_MinQoS = sr.evaluate_reachability("minimumQoS", delta=0.1, epsilon= 0.01)
  print(reach_MinQoS)

  reach_premQoS = sr.evaluate_reachability("premiumQoS", delta=0.1, epsilon= 0.01)

  print(reach_premQoS)

all aviable parameters name : 
['N', 'S', 'mf', 'mr']
N changed to 20current configuration : init
mf changed to 480current configuration : init
mr changed to 30current configuration : init
20.0
480.0
30.0
2.0
 The simulation has been successfully completed


 The reachability evaluation has been successfully completed

Probability of reaching minimumQoS is 
0.5886792452830188 

error prob (epsilon) :  0.01
error gap  (delta)   :  0.1

 The reachability evaluation has been successfully completed

Probability of reaching premiumQoS is 
0.5849056603773585 

error prob (epsilon) :  0.01
error gap  (delta)   :  0.1



In [ ]:
sibilla_runtime.clear()

**More no of Switches**

In [ ]:
with sibilla_runtime as sr:
  sr.load_module("population")
  sr.load_from_file("workstation_cluster.pm")
  sr.set_configuration("init")
  sr.set_deadline(1440)
  sr.set_dt(1)
  sr.set_replica(100)

  # get all the aviable parameters name
  print('all aviable parameters name : ')
  print(sr.get_parameters())

  sr.remove_all_measures()
  sibilla_runtime.add_measure("MinQoS")
  sibilla_runtime.add_measure("PremQoS")
  sibilla_runtime.add_measure("active_workstations")
  sibilla_runtime.add_measure("active_switches")
  sibilla_runtime.add_measure("active_backbone")
  sibilla_runtime.add_measure("repair_units_busy")
  sibilla_runtime.add_measure("repair_units_free")

  sr.set_parameter("S", 4, True, False)
  sr.set_configuration("init")

  print(sr.get_parameter('N'))
  print(sr.get_parameter('mf'))
  print(sr.get_parameter('mr'))
  print(sr.get_parameter('S'))

  simulation_result = sr.simulate("optimum_parameters")
  simulation_result.results = {"#"+k : v for k,v in simulation_result.results.items()}
  simulation_result.plot(show_sd = True)


all aviable parameters name : 
['N', 'S', 'mf', 'mr']
S changed to 4current configuration : init
5.0
120.0
40.0
4.0
 The simulation has been successfully completed


In [ ]:
sibilla_runtime.clear()

In [ ]:
sibilla_runtime.clear()

# **Optimal Solution**


**Factors that improves QoS:**
*   List item Redundent component

*   List item High MTBF




In [ ]:
with sibilla_runtime as sr:
  sr.load_module("population")
  sr.load_from_file("workstation_cluster.pm")
  sr.set_configuration("init")
  sr.set_deadline(1440)
  sr.set_dt(1)
  sr.set_replica(100)

  # get all the aviable parameters name
  print('all aviable parameters name : ')
  print(sr.get_parameters())

  sr.remove_all_measures()
  sibilla_runtime.add_measure("MinQoS")
  sibilla_runtime.add_measure("PremQoS")

  sr.set_parameter("N", 100, True, False)
  sr.set_parameter("mf", 720, True, False)
  sr.set_parameter("mr", 60, True, False)
  sr.set_configuration("init")

  print(sr.get_parameter('N'))
  print(sr.get_parameter('mf'))
  print(sr.get_parameter('mr'))
  print(sr.get_parameter('S'))

  simulation_result = sr.simulate("optimum_parameters")
  simulation_result.results = {"#"+k : v for k,v in simulation_result.results.items()}
  simulation_result.plot(show_sd = True)
  print(sr.evaluate_reachability("minimumQoS", delta=0.1, epsilon= 0.01))
  print(sr.evaluate_reachability("premiumQoS", delta=0.1, epsilon= 0.01))


all aviable parameters name : 
['N', 'S', 'mf', 'mr']
N changed to 100current configuration : init
mf changed to 720current configuration : init
mr changed to 60current configuration : init
100.0
720.0
60.0
2.0
 The simulation has been successfully completed


 The reachability evaluation has been successfully completed

Probability of reaching minimumQoS is 
0.47924528301886793 

error prob (epsilon) :  0.01
error gap  (delta)   :  0.1

 The reachability evaluation has been successfully completed

Probability of reaching premiumQoS is 
0.4641509433962264 

error prob (epsilon) :  0.01
error gap  (delta)   :  0.1

